# Fill sampletable/config.yaml for the chipseq-wf
Would be really useful to have a script to parse out information needed for these files because doing it by hand is a huge pain, and we have a lot of samples to run. 

All the information comes from this spreadsheet: 
{ note !! when saving this excel file, choose 'CSV UTF-8' } 


In [1]:
import pandas as pd
import glob
import os
import ruamel.yaml as YAML
import string

In [2]:
spreadsheet = pd.read_csv('../output/chip/20171103_s2cell_chip-seq.csv')
spreadsheet.head()

,srx,cell_line,geo,paper,strategy,antibody,srr,chromatin,input
0,SRX191907,S2 DRSC: Schneider's line 2,GSM1015404,"['24055367', '24055367']",ChIP-Seq,CP190 antibody (rabbit),SRR580337,0,SRX191915
1,SRX191908,S2 DRSC: Schneider's line 2,GSM1015405,"['24055367', '24055367']",ChIP-Seq,CP190 antibody (rabbit),SRR580338,0,SRX191916
2,SRX191909,S2 DRSC: Schneider's line 2,GSM1015406,"['24055367', '24055367']",ChIP-Seq,Su(Hw) antibody 406 (rabbit),SRR580339,0,SRX191915
3,SRX191910,S2 DRSC: Schneider's line 2,GSM1015407,"['24055367', '24055367']",ChIP-Seq,Su(Hw) antibody 406 (rabbit),SRR580340,0,SRX191916
4,SRX191911,S2 DRSC: Schneider's line 2,GSM1015408,"['24055367', '24055367']",ChIP-Seq,Mod(mdg4)2.2 antibody (rabbit),SRR580341,0,SRX191915


In [3]:
#For now we are excluding datasets with no input: 

spreadsheet = spreadsheet[spreadsheet.input != 'no input?']

### Sample table: 
- should have columns "samplename, antibody, biological_material, replicate, label, orig_filename" 
- put in a list of SRRs that you want sampletable for (for example all SRRs that aren't chromatin)
- write out to "sampletable_test.tsv" 

In [4]:
all_chromatin = spreadsheet[spreadsheet.chromatin == 1]
no_chromatin = spreadsheet[spreadsheet.chromatin == 0]

In [5]:
chrom_srr_list = all_chromatin.srr.values.tolist()
tf_srr_list = no_chromatin.srr.values.tolist()

In [6]:
SRR_list = tf_srr_list 
table = []
missing_files = []

def sanitize_fname(fname):
        valid_chars = "-_.%s%s" % (string.ascii_letters, string.digits)
        return ''.join([x for x in fname if x in valid_chars])

for val in SRR_list:
    for srr in val.split('|'):
        SRR = srr.strip()
        row = spreadsheet[spreadsheet.srr.astype(str).str.contains(SRR)]
        antibody = sanitize_fname(row.antibody.values[0])
        srx = row.srx.values[0]
        inpt = row.input.values[0]
        PATH = '/data/MiegNCBI/ncbi_remap/prealn-wf/output/samples/'+srx+'/'+SRR
        with open(glob.glob(PATH+'/LAYOUT')[0]) as f:
            for line in f:
                if os.path.isfile(PATH+'/'+SRR+'_1.fastq.gz'):
                    if [line == 'SE'] and [os.path.getsize(PATH+'/'+SRR+'_1.fastq.gz') > 0]: 
                        filename = PATH+'/'+SRR+'_1.fastq.gz'
                    if line == 'SE' and os.path.getsize(PATH+'/'+SRR+'_2.fastq.gz') > 0:
                        filename = PATH+'/'+SRR+'_2.fastq.gz'
                    if line == 'keep_R1':
                        filename = PATH+'/'+SRR+'_1.fastq.gz'
                    if line == 'keep_R2':
                        filename = PATH+'/'+SRR+'_2.fastq.gz'
                    if line == 'PE': 
                        filename = PATH+'/'+SRR+'_1.fastq.gz'
                        #decided to get rid of the 2nd
                        #row_2 = [SRR+'_2', antibody, 's2cell', '2', srx, PATH+'/'+SRR+'_2.fastq.gz']
                    new_row = [SRR, antibody, 's2cell', '1', srx, filename]
                    table.append(new_row)
                else:
                    myglob = glob.glob('/home/bergeric/data/s2cell-prior/output/chip/fastq_dump/'+SRR+'_*')
                    if myglob: 
                        filename = myglob[0]
                        new_row = [SRR, antibody, 's2cell', '1', srx, filename]
                        table.append(new_row)
                    else:
                        missing_files.append(SRR)
                                   
        if inpt != 'no input?':
            inpt_row = spreadsheet[spreadsheet.srx.astype(str).str.contains(inpt)]
            inpt_srr = inpt_row.srr.values[0]
            if inpt_srr not in SRR_list: 
                SRR_list.append(inpt_srr)
    
my_sampletable = pd.DataFrame(table, columns=['samplename','antibody','biological_material','replicate','label',
                                             'orig_filename']) 
#paired end only: 
#paired_end = my_sampletable[my_sampletable.samplename.astype(str).str.contains('_')]
#single end only:
#single_end = my_sampletable[~my_sampletable.samplename.astype(str).str.contains('_')]
#write out 
my_sampletable.to_csv('../chipseq-wf/config/sampletable_tf.tsv', sep='\t', index=False)
#single_end.to_csv('../chipseq-wf/config/sampletable_SE.tsv', sep='\t', index=False)
#paired_end.to_csv('../chipseq-wf/config/sampletable_PE.tsv', sep='\t', index=False)

In [7]:
missing_files

[]

In [8]:
my_sampletable.head()

,samplename,antibody,biological_material,replicate,label,orig_filename
0,SRR580337,CP190antibodyrabbit,s2cell,1,SRX191907,/data/MiegNCBI/ncbi_remap/prealn-wf/output/sam...
1,SRR580338,CP190antibodyrabbit,s2cell,1,SRX191908,/data/MiegNCBI/ncbi_remap/prealn-wf/output/sam...
2,SRR580339,SuHwantibody406rabbit,s2cell,1,SRX191909,/data/MiegNCBI/ncbi_remap/prealn-wf/output/sam...
3,SRR580340,SuHwantibody406rabbit,s2cell,1,SRX191910,/data/MiegNCBI/ncbi_remap/prealn-wf/output/sam...
4,SRR580341,Modmdg42.2antibodyrabbit,s2cell,1,SRX191911,/data/MiegNCBI/ncbi_remap/prealn-wf/output/sam...


### Config file
- yaml
- fill in label, algorithm, ip, control, extra
- write out to "copy_config.yaml"

In [9]:
#pull out labels from sample table that are not inputs  
#chromatin extra
#extra = '-g dm --bdg --broad --slocal 5000 --nomodel --extsize 147'
#tf extra
extra = '-g dm --bdg --nomodel --extsize 147'
SRX_list = set(my_sampletable.label.values)

block_list = []
with open('../chipseq-wf/config/config_tf.yaml', 'w') as outfile: 
    for SRX in SRX_list: 
        row = spreadsheet[spreadsheet.srx.astype(str).str.contains(SRX)]
        if not row.antibody.astype(str).str.contains('input').bool():
            label = SRX
            ip = SRX
            control = row.input.values[0]
            macs_block = {'label': label, 'algorithm': 'macs2', 'ip': [ip], 'control': [control], 
                                    'extra': YAML.scalarstring.SingleQuotedScalarString(extra)}
            spp_block = {'label': label, 'algorithm': 'spp', 'ip': [ip], 'control': [control], 
                                    'extra': {'fdr': 0.001}}
            if macs_block and spp_block not in block_list: 
                block_list.append(macs_block)
                block_list.append(spp_block)
    with open('../chipseq-wf/config/config.yaml', 'r') as c:
        page = YAML.load(c, Loader=YAML.RoundTripLoader, preserve_quotes=True)
        for block in block_list: 
            page['chipseq']['peak_calling'].append(block)
        YAML.dump(page, outfile, Dumper=YAML.RoundTripDumper)